# Batch Legal Mockup Model

In [5]:
#Imports

import pandas as pd
import string
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [7]:
!pwd

/home/christopher/code/fvanlitsenburg/BatchLegal/notebooks


In [8]:
#Loading html-file + converting it into txt

file = open("../raw_data/test_data.html", "r")
data = BeautifulSoup(file)
as_txt = data.get_text()

In [9]:
#Tokenizing the sentences

as_sentences = sent_tokenize(as_txt)


In [10]:
#Transforming sentences into DF

txt_df = pd.DataFrame(as_sentences)
txt_df

,0
0,\n\n\nL_2022020EN.01000101.xml\n\n\n\n\n\n\n\n...
1,(2)\n\n\nThe COVID-19 pandemic has highlighted...
2,As recognised by the World Health Organization...
3,Diseases may be transmitted from humans to ani...
4,"Approximately 70 % of emerging diseases, and a..."
...,...
560,(19) Regulation (EC) No 1049/2001 of the Euro...
561,(20) Directive (EU) 2019/1937 of the European...
562,(21) Directive (EU) 2016/943 of the European ...
563,"(22) Regulation (EU, Euratom) 2018/1046 of th..."


In [11]:
type(txt_df)

pandas.core.frame.DataFrame

In [19]:
ignore_list = {'ec', 'no', 'european', 'commission', 'eu', 'union',
                   'article', 'directive', 'council', 'regulation', 'official',
                   'journal', 'article', 'information', 'agency', 'regulation',
                   'mssg', 'data', 'member', 'states'
                  } # list used to remove unrelevant terms 

In [20]:
#Defining Davy's Preproc-Function

def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercasing 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    tokenized_sentence = word_tokenize(sentence) ## tokenizing 
    stop_words = set(stopwords.words('english')) ## defining stopwords
    tokenized_sentence_cleaned = [w for w in tokenized_sentence 
                                  if not w in stop_words] ## remove stopwords
    tokenized_sentence_cleaned = [w for w in tokenized_sentence_cleaned
                                  if not w in ignore_list]
    lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "v")  # v --> verbs
              for word in tokenized_sentence_cleaned]
    cleaned_sentence = ' '.join(word for word in lemmatized)
    return cleaned_sentence

In [21]:
# Applying Davy's Function

clean_txt = txt_df[0].apply(cleaning)

In [22]:
#Checking outcome of Preprocessing
clean_txt

0      lenxml en l parliament january reinforce role ...
1      covid pandemic highlight interconnectedness hu...
2      recognise world health organization many micro...
3      diseases may transmit humans animals vice vers...
4      approximately emerge diseases almost know pand...
                             ...                        
560    parliament may regard public access parliament...
561    parliament october protection persons report b...
562    parliament june protection undisclosed knowhow...
563    euratom parliament july financial rule applica...
564    february fee payable evaluation medicinal prod...
Name: 0, Length: 565, dtype: object

In [23]:
#Vectorizing data

vectorizer = CountVectorizer()
vectorized_text = vectorizer.fit_transform(clean_txt)

In [24]:
#Modelling

# Instantiating the LDA 
n_components = 3
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)

# Fitting the LDA on the vectorized documents
lda_model.fit(vectorized_text)

# Getting topics
topics = lda_model.transform(vectorized_text)

In [25]:
#Topic model function from ML-10-lecture
def print_topics(model, vectorizer, top_words):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], round(topic[i],2))
                        for i in topic.argsort()[:-top_words - 1:-1]])

In [26]:
#Printing topics

print_topics(lda_model, vectorizer, top_words = 8)

--------------------
Topic 0:
[('health', 114.16), ('medicinal', 109.44), ('products', 108.94), ('public', 72.78), ('emergencies', 51.14), ('shortages', 48.6), ('medical', 44.35), ('parliament', 43.26)]
--------------------
Topic 1:
[('medicinal', 148.16), ('refer', 133.16), ('shall', 121.71), ('products', 106.68), ('point', 95.23), ('list', 84.29), ('include', 77.33), ('critical', 75.78)]
--------------------
Topic 2:
[('devices', 106.54), ('shall', 93.93), ('medical', 80.92), ('provide', 64.89), ('etf', 60.27), ('clinical', 56.22), ('mdssg', 52.56), ('representatives', 42.59)]
